### Analyze whether TOBS varies more from year to year or from station to station.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions


In [2]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSSSBSBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1303
+------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-----------------+------------------+---------+--------+--------+---------+-----------+------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-----------+----------+------+--------------------+------+
|           coeff_1|          coeff_10|           coeff_11|           coeff_12|           coeff_13|          coeff_14|           coeff_15|           coeff_16|            coeff_2

In [3]:
print df.columns

['coeff_1', 'coeff_10', 'coeff_11', 'coeff_12', 'coeff_13', 'coeff_14', 'coeff_15', 'coeff_16', 'coeff_2', 'coeff_3', 'coeff_4', 'coeff_5', 'coeff_6', 'coeff_7', 'coeff_8', 'coeff_9', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_10', 'res_11', 'res_12', 'res_13', 'res_14', 'res_15', 'res_16', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [4]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00011300,1957.0,-219.953018
1,USC00013816,1961.0,-21.509786
2,USC00018323,1985.0,360.671411
3,USC00012245,1966.0,50.747606
4,USC00017366,1975.0,-176.706376


In [5]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00010616,USC00011069,USC00011189,USC00011300,USC00011301,USC00012188,USC00012240,USC00012245,USC00013024,USC00013308,...,USC00018178,USC00018323,USC00018398,USC00018403,USC00018438,USC00018446,USC00018859,USC00018862,USC00018867,USW00013821
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-539.465002,NaN,325.203952,NaN,NaN,-235.682128,53.869825,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260.691406,NaN,...,NaN,NaN,287.827969,NaN,NaN,293.314869,404.800568,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-58.168401,NaN,NaN,-45.417893,-100.105880,NaN,NaN,NaN
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-191.242254,74.568024,NaN,NaN,NaN
1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,97.872213,232.681353,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-27.317797,-0.204355,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-501.962357,-322.614256,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,110.155643,459.983873,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-128.493508,NaN,NaN,NaN,-61.722446,-119.066236,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [6]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                    = ',RMS(year_station_table)
print 'RMS removing mean-by-station = ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year    = ',RMS(tbl_minus_year)

total RMS                   =  192.931081902
RMS removing mean-by-station=  189.319527345
RMS removing mean-by-year   =  104.605404872


In [8]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(10):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year     =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations =',RMS(T)

initial RMS= 192.931081902
0 after removing mean by year     = 104.605404872
0 after removing mean by stations = 99.5261923657
1 after removing mean by year     = 99.3453902518
1 after removing mean by stations = 99.3116040689
2 after removing mean by year     = 99.3001023212
2 after removing mean by stations = 99.2953353876
3 after removing mean by year     = 99.2932068539
3 after removing mean by stations = 99.2922238056
4 after removing mean by year     = 99.2917620307
4 after removing mean by stations = 99.2915431789
5 after removing mean by year     = 99.2914389634
5 after removing mean by stations = 99.2913892099
6 after removing mean by year     = 99.2913654244
6 after removing mean by stations = 99.291354045
7 after removing mean by year     = 99.2913485986
7 after removing mean by stations = 99.2913459913
8 after removing mean by year     = 99.291344743
8 after removing mean by stations = 99.2913441453
9 after removing mean by year     = 99.2913438591
9 after removing mean by 